<a href="https://colab.research.google.com/github/jjonhwa/Hands-On-Machine-Learning/blob/main/Chapter5_%EC%84%9C%ED%8F%AC%ED%8A%B8_%EB%B2%A1%ED%84%B0_%EB%A8%B8%EC%8B%A0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**SVM**
1. 선형, 비선형, 회귀, 이상치 탐색에도 사용할 수 있는 다목적 머신러닝 모델
2. 특히, 복잡한 분류 문제, 작거나 중간 크기의 데이터셋일 때 적합하다.

# 5.1 선형 SVM 분류
선형 분류 : 두 클래스를 나누고 있을 뿐만 아니라 제일 가까운 훈련 샘플로부터 가능한 한 멀리 떨어져 있는 직선을 구한다.(라지 마진 분류 : 가장 폭이 넓은 도로를 찾는다.)  

**Note** : SVM은 스케일에 민감하다.(`StandardScaler`를 사용하여 스케일을 조정하도록 하자.)  
* 하드 마진 분류
1. 모든 샘플이 도로 바깥쪽에 올바르게 분류된 경우
2. 데이터가 선형적으로 구분될 수 있어야 사용할 수 있음.
3. 이상치에 민감함.

* 소프트 마진 분류
1. 도로 폭을 가능한 넓게 유지하면서 마진 오류를 작게한다.
2. SVM모델에서 하이퍼 파라미터 `C`를 이용하여 폭을 조정할 수 있다.(`C`가 작을수록 폭이 넓어진다.)

**Note** : SVM을 사용할 때 Overfitting이 됬을 경우 하이퍼 파라미터 `C`를 감소시켜 모델을 규제할 수 있다.

In [1]:
import numpy as np
from sklearn import datasets
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC

iris = datasets.load_iris()
X = iris['data'][:, (2,3)]
y = (iris['target'] == 2).astype(np.float64)

svm_clf = Pipeline([
      ('scaler', StandardScaler()),
      ('linear_svc', LinearSVC(C = 1, loss = 'hinge')),
])

svm_clf.fit(X, y)

Pipeline(memory=None,
         steps=[('scaler',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('linear_svc',
                 LinearSVC(C=1, class_weight=None, dual=True,
                           fit_intercept=True, intercept_scaling=1,
                           loss='hinge', max_iter=1000, multi_class='ovr',
                           penalty='l2', random_state=None, tol=0.0001,
                           verbose=0))],
         verbose=False)

In [2]:
svm_clf.predict([[5.5, 1.7]])

array([1.])

* 선형 SVM 모델을 사용하는 방법(분류)
1. LinearSVC 사용
2. SVC에서 `kernel = 'linear'`로 준다.
3. SGDClassifier를 이용한다. (`SGDClassifier(loss = 'hinge', alpha = 1/(m*C))`를 이용한다.) (SGDClassifier의 경우 빠르게 수렴하진 않지만 데이터 셋이 아주 커서 메모리에 적재할 수 없거나 온라인 학습으로 분류 문제를 다룰 때 유용하다.)

**Note** : LinearSVC의 경우 규제에 편향이 포함되어 있어서 평균을 뺴서 중앙에 맞도록 해주어야 한다. 그래서 StandardScaler를 써서 맞춰 주도록 하며 loss의 경우 'hinge'로 지정해주도록 한다. 또한, 특성이 샘플보다 적다면 `dual`매개변수를 False로 지정해주어야 한다.(duality(쌍대) 문제)

# 5.2 비선형  SVM 분류
비선형을 만드는 방법을 알아보도록 하고 기본적으로 Pipeline 에 다항 특성을 추가해서 구성하는 방법이 있다.

In [3]:
from sklearn.datasets import make_moons
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures

X, y = make_moons(n_samples = 100, noise = 0.15)
polynomial_svm_clf = Pipeline([
      ('poly_features', PolynomialFeatures(degree = 3)),
      ('scaler', StandardScaler()),
      ('svm_clf', LinearSVC(C = 10, loss = 'hinge'))
])

polynomial_svm_clf.fit(X, y)

/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


Pipeline(memory=None,
         steps=[('poly_features',
                 PolynomialFeatures(degree=3, include_bias=True,
                                    interaction_only=False, order='C')),
                ('scaler',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('svm_clf',
                 LinearSVC(C=10, class_weight=None, dual=True,
                           fit_intercept=True, intercept_scaling=1,
                           loss='hinge', max_iter=1000, multi_class='ovr',
                           penalty='l2', random_state=None, tol=0.0001,
                           verbose=0))],
         verbose=False)

## 5.2.1 다항식 커널
SVM을 사용할 때 커널 트릭을 이용한다.
* 커널 트릭(kernel trick) : 실제로는 특성을 추가하지 않으면서 다항식 특성을 많이 추가한 것과 같은 결과를 얻는 방법  



In [4]:
from sklearn.svm import SVC
poly_kernel_svm_clf = Pipeline([
      ('scaler', StandardScaler()),
      ('svm_clf', SVC(kernel = 'poly', degree = 3, coef0 = 1, C = 5))
])

poly_kernel_svm_clf.fit(X, y)

Pipeline(memory=None,
         steps=[('scaler',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('svm_clf',
                 SVC(C=5, break_ties=False, cache_size=200, class_weight=None,
                     coef0=1, decision_function_shape='ovr', degree=3,
                     gamma='scale', kernel='poly', max_iter=-1,
                     probability=False, random_state=None, shrinking=True,
                     tol=0.001, verbose=False))],
         verbose=False)

위의 코드는 '3차 다항식 커널'을  사용해 SVM 분류기를 훈련시킨다.  

* 모델이 과대적합이라면 다항식의 차수를 줄여야한다.  

* 매개변수 `coef0`은 모델이 높은 차수와 낮은 차수에 얼마나 영향을 받을지를 조절한다.  

**Note** : 적절한 하이퍼파라미터를 찾는 일반적인 방법은 GridSearch를 이용하는 것이다. 처음에는 Grid의 폭을 크게하여 빠르게 검색하고 그 후에는 최적 값을 찾기 위해 Grid를 세밀하게 검색하여 사용하도록 한다.


## 5.2.2 유사도 특성
비선형 특성을 다루는 또 다른 기법은 각 샘플이 특정 랜드마크와 얼마나 닮았는지 측정하는 '유사도 함수'로 계산한 특성을 추가하는 것이다.

* 유사도 함수 : 각 샘플에 대해 특정 랜드마크와의 유사도를 측정하는 함수  


* 유사도 함수 예제 : 가우시안 방사 기저 함수(RBF, Radial Basis Function)
1. $l$ : 랜드마크
2. $\gamma$ : 랜드마크에서 멀어질수록 0에 수렴하는 속도를 조절함
  - $\gamma$값이 클수록 가까운 샘플선호
  - 과대적합 위험 커짐
  - 0: 랜드마크에서 아주 멀리 떨어진 경우
  - 1: 랜드마크와 같은 위치인 경우

가우시안 RBF : $\phi_\gamma (x, l)$ = $exp(-\gamma\| x-l\|^2)$

* 유사도 함수 적용 장단점
1. 각 샘플을 랜드마크로 지정 후 유사도 특성을 추가하면 (n개의 특성을 가진 m개의 샘플)에서 (m개의 특성을 가진 m개의 샘플)이 된다.
2. 장점 : 차원이 커지면서 선형적으로 구분될 가능성이 높아짐.
3. 단점 : 훈련 세트가 매우 클 경우 동일한 크기의 아주 많은 특성이 생성됨.

## 5.2.3 가우시안 RBF 커널

다항 특성 방식과 마찬가지로 유사도 특성 방식도 머신러닝 알고리즘에 유용하게 사용될 수 있다.  

추가 특성을 모두 계산하려면 연산 비용이 많이 드는데 특히 훈련 세트가 클 경우 더 그렇다.  

다음은 가우시안 RBF 커널을 사용한 SVC 모델이다 확인해보도록 하자.


In [5]:
rbf_kernel_svm_clf = Pipeline([
      ('scaler', StandardScaler()),
      ('svm_clf', SVC(kernel = 'rbf', gamma = 5, C = 0.001))
])
rbf_kernel_svm_clf.fit(X, y)

Pipeline(memory=None,
         steps=[('scaler',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('svm_clf',
                 SVC(C=0.001, break_ties=False, cache_size=200,
                     class_weight=None, coef0=0.0,
                     decision_function_shape='ovr', degree=3, gamma=5,
                     kernel='rbf', max_iter=-1, probability=False,
                     random_state=None, shrinking=True, tol=0.001,
                     verbose=False))],
         verbose=False)

1. $\gamma$를 증가시키면 종 모양 그래프가 조ㅗㅂ아져서 각 샘플의 영향 범위가 작아진다.
2. $\gamma$가 작으면 넓은 종 모양 그래프를 만들며 샘플이 넒은 범위에 걸쳐 영향을 주므로 경계가 더 부드러워 진다.
3. 즉, $\gamma$는 규제의 역할을 한다.(모델이 Overfit일 경우 감소시켜야 한다.) 

**Note**
* 여러 가지 커널 중 어떤 것을 사용할지 모를 때는 경험적으로 봤을 떄 **선형 커널**을 가장 먼저 시도해보도록 하자. 
* `LinearSVC`가 `SVC(kernel = 'linear'`보다 훨씬 바르다는 것을 기억하도록 하자(Training Set이 아주 크거나 특성 수가 많을 경우 더욱 그렇다). 
* Training Set이 너무 크지 않다면 가우시안 RBF 커널도 시도해볼만 하다. 
* 시간과 컴퓨팅 성능이 충분하다면 cross validation & grid search도 사용하여 다른 커널들도 더 시도해보도록 하자.

## 5.2.4 계산 복잡도
Linear SVC 파이썬 클래스틑 선형 SVM을 위한 최적화된 알고리즘을 구현한 liblinear 라이브러리를 기반으로 한다. liblinear 라이브러리는 kernel trick을 지원하지 않지만 훈련 샘플과 특성 수에 거의 선형적으로 늘어난다. 이 알고리즘의 훈련 시간 복잡도는  대략 mxn 정도이다.(SVC는 libsvm 라이브러리 기반)

* SVC의 경우 훈련 시간의 복잡도는 보통 $m^2$ x $n$과 $m^3$ x $n$사이이다.  
* **이는 샘플 수가 늘어날수록 엄청나게 느려진다는 것을 의미하며 이로 인해 SVC의 경우 복잡하지만 작거나 중간 규모의 훈련 세트를 사용할 떄 알맞다.**
* 희소 특성(각 샘플에 0이 아닌 특성이 몇개 없는 경우)인 경우에 잘 확장된다.(알고리즘의 성능이 샘플이 가진 0이 아닌 특성의 평균 수에 거의 비례한다.)

# 5.3 SVM 회귀
* SVM을 분류가 아니라 회귀에 적용하는 방법은 목표를 반대로 하는 것이다.
* 제한된 마진 오류(도로 밖의 샘플) 안에서 도로 안에 가능한 한 많은 샘플이 들어가도록 학습한다.(회귀에서 도로의 폭은 epsilon으로 조절한다.)
* 선형 SVM 회귀의 경우 마진 안에 훈련 샘플이 추가되어도 모델의 예측에 영향이 없어서 **$\epsilon$에 민감하지 않다**라고 말한다.

다음의 코드는 `LinearSVR`을 사용한 선형 SVM 회귀이다.

In [6]:
from sklearn.svm import LinearSVR

svm_reg = LinearSVR(epsilon = 1.5)
svm_reg.fit(X, y)

LinearSVR(C=1.0, dual=True, epsilon=1.5, fit_intercept=True,
          intercept_scaling=1.0, loss='epsilon_insensitive', max_iter=1000,
          random_state=None, tol=0.0001, verbose=0)

비선형 회귀 작업의 경우 '커널 SVM 모델'을 사용한다.(여기서 역시 C로 규제를 정할 수 있다.)   
다음은 `SVR`을 사용한 비선형 SVM 회귀이다.

In [7]:
from sklearn.svm import SVR

svm_poly_reg = SVR(kernel = 'poly', degree = 2, C = 100, epsilon = 0.1)
svm_poly_reg.fit(X, y)

SVR(C=100, cache_size=200, coef0=0.0, degree=2, epsilon=0.1, gamma='scale',
    kernel='poly', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

LinearSVR은 LinearSVC와 마찬가지로 필요한 시간이 Training Set에 비례해서 선형적으로 늘어나며 SVR은 Training Set이 커질수록 훨씬 느려진다.